In [4]:
import requests
import pandas as pnd

df = pnd.read_csv("Wallet_id-Sheet1.csv")
wallet_ids = df.iloc[:, 0].dropna().tolist()

def score_wallet(wallet):
    url = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"
    query = """
    {
  account(id: "%s") {
    id
    tokens {
      symbol
      storedBorrowBalance
      totalUnderlyingSupplied
    }
  }
}
    """ % wallet.lower()
    response = requests.post(url, json={'query': query})
    data = response.json()
    account = data.get("data",{}).get('account')
    if not account:
        return 0
        
    total_supply = sum([float(t.get("totalUnderlyingSupplied", 0)) for t in account["tokens"]])
    total_borrow = sum([float(t.get("storedBorrowBalance", 0)) for t in account["tokens"]])
    if total_supply + total_borrow == 0:
        return 0

    ratio = total_borrow / (total_supply + total_borrow)
    score = int(ratio * 1000)
    return min(score, 1000)

results = []
for wallet in wallet_ids:
    print(f"Scoring {wallet} ...")
    score = score_wallet(wallet.lower())
    results.append({"wallet_id": wallet, "score": score})

output_df = pnd.DataFrame(results)
output_df.to_csv("WalletRiskScores.csv", index=False)
print("Risk scoring complete. Result saved to WalletRiskScore.csv")

Scoring 0x0039f22efb07a647557c7c5d17854cfd6d489ef3 ...
Scoring 0x06b51c6882b27cb05e712185531c1f74996dd988 ...
Scoring 0x0795732aacc448030ef374374eaae57d2965c16c ...
Scoring 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9 ...
Scoring 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae ...
Scoring 0x104ae61d8d487ad689969a17807ddc338b445416 ...
Scoring 0x111c7208a7e2af345d36b6d4aace8740d61a3078 ...
Scoring 0x124853fecb522c57d9bd5c21231058696ca6d596 ...
Scoring 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc ...
Scoring 0x1656f1886c5ab634ac19568cd571bc72f385fdf7 ...
Scoring 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22 ...
Scoring 0x19df3e87f73c4aaf4809295561465b993e102668 ...
Scoring 0x1ab2ccad4fc97c9968ea87d4435326715be32872 ...
Scoring 0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f ...
Scoring 0x1e43dacdcf863676a6bec8f7d6896d6252fac669 ...
Scoring 0x22d7510588d90ed5a87e0f838391aaafa707c34b ...
Scoring 0x24b3460622d835c56d9a4fe352966b9bdc6c20af ...
Scoring 0x26750f1f4277221bdb5f6991473c6ece8c821f9d ...
Scoring 0x